In [2]:
#importing the required libraries
import numpy as np
import pandas as pd

In [3]:
#loading the MNIST fashion dataset 
from keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

#storing the training data in a dataframe
train_df = pd.DataFrame(np.hstack([x_train.reshape(-1, 28*28),y_train.reshape(-1,1)]))

#saving the dataframe as csv file
train_df.to_csv('train.csv', index=False, header=False)

In [4]:
#building a GAN
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

#defining the size of the latent space
latent_dim = 100

In [5]:
#defining the discriminator model
discriminator = Sequential()
discriminator.add(Flatten(input_shape=(28, 28, 1)))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

#defining the generator model
generator = Sequential()
generator.add(Dense(256, input_dim=latent_dim))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(512))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(1024))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(28*28, activation='tanh'))
generator.add(Reshape((28, 28, 1)))

#combining the generator and the discriminator
gan_input = Input(shape=(latent_dim,))
x = generator(gan_input)
gan_output = discriminator(x)
gan = Model(inputs=gan_input, outputs=gan_output)

#compiling the model
optimizer = Adam(lr=0.0002, beta_1=0.5)
gan.compile(loss='binary_crossentropy', optimizer=optimizer)

C:\Users\Surya\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [6]:
#generating artificial data for 100 examples
noise = np.random.normal(0, 1, (100, latent_dim))
generated_images = generator.predict(noise)

#storing the artificial data in dataframe
test_df = pd.DataFrame(generated_images.reshape(100, 28*28))

#saving the dataframe as csv file
test_df.to_csv('test.csv', index=False, header=False)

4/4 [==============================] - 1s 8ms/step
